In [1]:
#Import Modules
import numpy as np
import pandas as pd
import pickle

print("Modules Imported Successfully!")

Modules Imported Successfully!


In [2]:
#Loading the Dataset
df = pd.read_csv('capitals.txt', delimiter=' ')

In [3]:
#Renaming the column names
df.columns=["city1", "country1", "city2", "country2"]
df.columns

Index(['city1', 'country1', 'city2', 'country2'], dtype='object')

In [4]:
df.head()

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


In [5]:
word_embeddings = pickle.load(open("word_embeddings_subset.p", "rb"))

In [6]:
len(word_embeddings)

243

In [7]:
#function to calculate cosine similarity
def cosine_similarity(A, B):
    """sum = 0 #Initialising the sum to 0
    for x,y in zip(A,B): #Iterating on both the lists together
        sum += x*y #adding sum and product of each element at ith poisition
    norm_a, norm_b = np.linalg.norm(A), np.linalg.norm(B) #calculating norm of A and B 
    cos_b = sum/(norm_a*norm_b) #calculating cos(beta)
    return cos_b"""
    
    dot = np.dot(A,B) #dot product of A & B
    norm_a = np.sqrt(np.dot(A,A)) #magnitude of A
    norm_b = np.sqrt(np.dot(B,B)) #magnitude of B
    return dot/(norm_a*norm_b)

In [8]:
#testing our cosine similarity function
king = word_embeddings["king"] #getting the word embeddings for the word
queen = word_embeddings["queen"]

cosine_similarity(king, queen) #calculating the cosine similarity between the words

0.6510956

In [9]:
#getting the country of inputs
def get_country(city1, country1, city2, word_embeddings):
    
    #getting the word embeddings for our input
    city1_emb = word_embeddings[city1] 
    country1_emb = word_embeddings[country1]
    city2_emb = word_embeddings[city2]
    
    vec = country1_emb - city1_emb + city2_emb #getting the embeddings for our prediction 
    
    similarity = -1 #initialising the similarity to -1
    country = '' #initialising an empty string for country
    
    group = set((city1, country1, city2)) #input set
    
    for word in word_embeddings.keys(): #iterating through words in embedddings
        
        if word not in group: #in order to not get any input country as output
            word_emb = word_embeddings[word] #get the word embeddings
            cos_similarity = cosine_similarity(vec, word_emb) #getting the cosine similarity for the word vector and predicted vectore
            
            if cos_similarity > similarity: #if current word similarity is greater
                similarity = cos_similarity #new similarity => current similarity
                country = (word, similarity) #changing country to (country, similarity) pair
                
    return country
    

In [10]:
get_country('Athens', 'Greece', 'Bern', word_embeddings) #checking our get_country function

('Switzerland', 0.5022102)

In [11]:
def get_accuracy(word_embeddings, data):
    num_correct = 0 #to count the right predictions
    
    for i, row in data.iterrows(): #for iterating through rows of our data
        
        city1 = row["city1"] #getting the city1 column
        country1 = row["country1"]
        city2 = row["city2"]
        country2 = row["country2"] #output country
        
        country2_pred, _ = get_country(city1, country1, city2, word_embeddings) #get country for current group
        
        if country2_pred == country2: #if predicted country is equal to actual country
            num_correct += 1 #increase the nnumber of accurate predictions
        
    m = len(data) #total length of data
    accuracy = num_correct/m #calculating accuracy
    return accuracy

In [12]:
print(round(get_accuracy(word_embeddings, df),4)*100, "%") #getting the accuracy of our predictor

91.92 %
